In [1]:
import os, sys, io, random, subprocess
import string
import numpy as np
import pandas as pd

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO

sys.path.append('/home/tortoise/pybioviz/')
from pybioviz import viewers, utils

In [33]:
featurekeys = ['ID', 'Name', 'del_len', 'ins_len', 'query_coord', 'start', 'end']

def features_to_dataframe(features, cds=False):
    """Get features from a biopython seq record object into a dataframe
    Args:
        features: bio seqfeatures
       returns: a dataframe with a row for each cds/entry.
      """

    #preprocess features
    allfeat = []
    for (item, f) in enumerate(features):
        x = f.__dict__
        q = f.qualifiers
        #print(q)
        x.update(q)
        d = {}
        d['start'] = f.location.start
        d['end'] = f.location.end
        #d['strand'] = f.location.strand
        for i in featurekeys:
            if i in x:
                if type(x[i]) is list:
                    d[i] = x[i][0]
                else:
                    d[i] = x[i]
        allfeat.append(d)

    df = pd.DataFrame(allfeat,columns=featurekeys)

    return df

feats = utils.gff_to_features('ref_genomes/Nucdiff_MTB_MB/results/MTB_MB_ref_struct.gff')
feat_df = features_to_dataframe(feats)
#print(feat_df)
deletions = feat_df[feat_df.Name=='deletion']
deletions[:5]

,ID,Name,del_len,ins_len,query_coord,start,end
5,SV_6,deletion,1902,NaN,264979,264754,266656
7,SV_7,deletion,375,NaN,332982,334658,335033
21,SV_22,deletion,21,NaN,838696,836872,836893
28,SV_29,deletion,1355,NaN,891209,889020,890375
30,SV_31,deletion,111,NaN,928985,928092,928203


In [26]:
RD = pd.read_csv('RD.csv', sep='\t')
print(RD)
new=[]
for i,r in RD.iterrows():
    if r.Start>3.0e6 and r.Start<4e6:
        new.append(r)        
new = pd.DataFrame(new)

    No.   RD targets    Start     Stop  Size (bp)       Rv in RD
0     1          RD9  2330000  2332100       2029  Rv2073c-2075c
1     2        RD711  1501713  1503655       1943    Rv1333-1336
2     3        RD702   216795   218516       1722         Rv0186
3     4          RD4  1696200  1708800      12732  Rv1506c-1516c
4     5       RD1bcg  4349600  4359000       9400   Rv3871-3879c
5     6       RD1mic  4340400  4353500      13100    Rv3864-3871
6     7      RD2seal  2221000  2222900       1901   Rv1978-1979c
7     8       RD2bcg  2221100  2231800      10701    Rv1978-1988
8     9          RD7  2208000  2220500      12719    Rv1964-1977
9    10          RD8  4056800  4062600       5894    Rv3617-3623
10   11         RD10   264700   266600       1902    Rv0221-0223
11   12      RD12bov  3484700  3487500       2801    Rv3117-3121
12   13      RD12can  3479400  3491800      12401   Rv3111-3126c
13   14        RD105    79567    83034       3468    Rv0071-0074
14   15        RD239  409

In [15]:
new

,No.,RD targets,Start,Stop,Size (bp),Rv in RD
11,12,RD12bov,3484700,3487500,2801,Rv3117-3121
12,13,RD12can,3479400,3491800,12401,Rv3111-3126c
19,20,RD207,3120521,3127920,7400,Rv2814c-2820c
26,27,RD219,3448504,3451396,2893,Rv3083-3085
28,29,RD726,3904958,3906706,1749,Rv3485c-3487c
30,31,7bp pks15/1,3296380,3296381,7,Rv2946c-2947c


In [67]:
print(deletions)
print(RD)

         ID      Name del_len ins_len query_coord    start      end
5      SV_6  deletion    1902     NaN      264979   264754   266656
7      SV_7  deletion     375     NaN      332982   334658   335033
21    SV_22  deletion      21     NaN      838696   836872   836893
28    SV_29  deletion    1355     NaN      891209   889020   890375
30    SV_31  deletion     111     NaN      928985   928092   928203
37    SV_38  deletion      81     NaN     1306938  1305537  1305618
39    SV_40  deletion    3000     NaN     1404209  1402937  1405937
45    SV_46  deletion    1353     NaN     1544395  1541951  1543304
47    SV_48  deletion     336     NaN     1573590  1572502  1572838
57    SV_57  deletion     806     NaN     1758654  1768073  1768879
65    SV_65  deletion     399     NaN     1992301  1997053  1997452
66    SV_66  deletion      26     NaN     2069394  2074521  2074547
68    SV_68  deletion     123     NaN     2083613  2088852  2088975
82    SV_81  deletion    2028     NaN     231839

In [75]:
del3e = []
for i,r in RD.iterrows():    
    x = deletions[((deletions.start>r.Start) & (deletions.start<r.Stop)) |
              ((deletions.end>r.Start) & (deletions.end<r.Stop)) |
              ((deletions.start<r.Start) & (deletions.end>r.Stop))]
    del3e.append(x)
del3e = pd.concat(del3e)
print(del3e)

         ID      Name del_len ins_len query_coord    start      end
82    SV_81  deletion    2028     NaN     2318399  2330073  2332101
177  SV_175  deletion    5893     NaN     4004383  4056840  4062733
5      SV_6  deletion    1902     NaN      264979   264754   266656
121  SV_119  deletion    2774     NaN     3445699  3484740  3487514
121  SV_119  deletion    2774     NaN     3445699  3484740  3487514
96    SV_95  deletion    1353     NaN     2533378  2550013  2551366


ValueError: Can only compare identically-labeled Series objects